### Exploration:

Objective: Focus on trying to extract the employer from a list of `PushEvent` (Github commits). Script doesn't need to deal with deduplication.

Iterative process:

Input: a json (pandas row) of type Github PushEvent, where the relevant column is "payload"

Output: employer match, or None if not found

Deduplication at the end.

In [39]:
import pandas as pd
import json
import re
from github import Github
import os
import pprint
pp = pprint.PrettyPrinter()


In [5]:
# df = pd.read_gbq("""SELECT * FROM `gcp-cset-projects.cset_intern_jms727.light_archive`""") # gcp-cset-projects
# df.to_hdf("light_archive.hdf", key="a", index=False)
df = pd.read_hdf("light_archive.hdf", key="a")

In [6]:
# df["type"].value_counts()
pushEvent = df.loc[df["type"] == "PushEvent"]
pushEvent = pushEvent.drop_duplicates(subset=["other"])
# payload = pushEvent["payload"]

In [22]:
pushEvent["actor"].iloc[0]["login"]

'rsanheim'

In [7]:
test = pushEvent["payload"].iloc[0]
jsp = json.loads(test)
jsp

{'push_id': 8999348580,
 'size': 1,
 'distinct_size': 1,
 'ref': 'refs/heads/master',
 'head': 'c93a79f403ac9669d971a089354b362d5a861df7',
 'before': '8f7c02e681316c711b6ddd68392f084e4b8d539d',
 'commits': [{'sha': 'c93a79f403ac9669d971a089354b362d5a861df7',
   'author': {'email': 'd272b771ac5801e6f4a8185e9669dd091ae19b71@gmail.com',
    'name': 'Rob Sanheim'},
   'message': "BD help contact - take two (#3348)\n\n* Added Shamim's contact for HELP in BD\r\n\r\n* Added CSS for Bangladesh help button\r\n\r\n* Fix the help link\r\n\r\n* Add properly encoded URL for BD whatsapp support link\r\n\r\nCo-authored-by: Daniel Burka <danielburka@gmail.com>\r\nCo-authored-by: Tim Cheadle <tcheadle@resolvetosavelives.org>\r\nCo-authored-by: Hari Mohanraj <hari.mohanraj89@gmail.com>",
   'distinct': True,
   'url': 'https://api.github.com/repos/simpledotorg/simple-server/commits/c93a79f403ac9669d971a089354b362d5a861df7'}]}

In [8]:
# attempt to find email addresses in pre-json string
re.findall("([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)", test)

['d272b771ac5801e6f4a8185e9669dd091ae19b71@gmail.com',
 'danielburka@gmail.com',
 'tcheadle@resolvetosavelives.org',
 'hari.mohanraj89@gmail.com']

In [18]:
# Instantiate Github API
g = Github(os.environ["GITHUB_ACCESS_KEY"])


In [25]:
# Now ping the GitHub API for the user
username = pushEvent["actor"].iloc[0]["login"]
firstuser = g.get_user(username)


In [32]:
firstuser.raw_data

{'login': 'rsanheim',
 'id': 69,
 'node_id': 'MDQ6VXNlcjY5',
 'avatar_url': 'https://avatars.githubusercontent.com/u/69?v=4',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/rsanheim',
 'html_url': 'https://github.com/rsanheim',
 'followers_url': 'https://api.github.com/users/rsanheim/followers',
 'following_url': 'https://api.github.com/users/rsanheim/following{/other_user}',
 'gists_url': 'https://api.github.com/users/rsanheim/gists{/gist_id}',
 'starred_url': 'https://api.github.com/users/rsanheim/starred{/owner}{/repo}',
 'subscriptions_url': 'https://api.github.com/users/rsanheim/subscriptions',
 'organizations_url': 'https://api.github.com/users/rsanheim/orgs',
 'repos_url': 'https://api.github.com/users/rsanheim/repos',
 'events_url': 'https://api.github.com/users/rsanheim/events{/privacy}',
 'received_events_url': 'https://api.github.com/users/rsanheim/received_events',
 'type': 'User',
 'site_admin': False,
 'name': 'Rob Sanheim',
 'company': 'Simple.org',
 'blog': '

In [42]:
print(firstuser.bio)
print(firstuser.company)
print(firstuser.location)
print(firstuser.blog)
pp.pprint(firstuser.raw_data)
re.findall("\@\s?(.*?)\s", firstuser.bio)

Lead engineer @ Simple.org; formerly @ GitHub, Cognitect, and others. Human centered software development.
Simple.org
Madison, WI, NORTH AMERICA
https://rsanheim.com
{'avatar_url': 'https://avatars.githubusercontent.com/u/69?v=4',
 'bio': 'Lead engineer @ Simple.org; formerly @ GitHub, Cognitect, and others. '
        'Human centered software development.',
 'blog': 'https://rsanheim.com',
 'company': 'Simple.org',
 'created_at': '2008-01-27T07:09:47Z',
 'email': None,
 'events_url': 'https://api.github.com/users/rsanheim/events{/privacy}',
 'followers': 180,
 'followers_url': 'https://api.github.com/users/rsanheim/followers',
 'following': 28,
 'following_url': 'https://api.github.com/users/rsanheim/following{/other_user}',
 'gists_url': 'https://api.github.com/users/rsanheim/gists{/gist_id}',
 'gravatar_id': '',
 'hireable': None,
 'html_url': 'https://github.com/rsanheim',
 'id': 69,
 'location': 'Madison, WI, NORTH AMERICA',
 'login': 'rsanheim',
 'name': 'Rob Sanheim',
 'node_id':

['Simple.org;', 'GitHub,']